In [ ]:
import pynamix as px
import tifffile as tiff
import matplotlib.pyplot as plt
import numpy as np

side=np.array(tiff.imread('SurfaceEstimation_D0.tif')) ; 
baseline = 740 #px 
scale = 25/85 #mm/px
elevation=px.measure.surface_elevation_sideview(side, scale, baseline, threshold=-1)
plt.imshow(side)
plt.plot(elevation[0,:]/scale, baseline-elevation[1,:]/scale)

In [ ]:
# Preparing the arrays for fitting
top=np.array(tiff.imread('SurfaceEstimation_D1.tif')) ; 
top_flat=np.array(tiff.imread('SurfaceEstimation_D1_flat.tif')) ; 
scale_D1=101/440 #mm/px
center=194+(619-194)//2

#plt.imshow(top)
#plt.plot((194+(619-194)//2)*np.ones(960), range(0, 960))
D0_range=[134,14+32] ;
D1_range=[12+32,810] ;

#plt.plot(np.arange(0, D1_range[1]-D1_range[0])*scale_D1, np.log(top[D1_range[0]:D1_range[1],center]/top_flat[D1_range[0]:D1_range[1],center]))
#plt.plot(elevation[0, :-D0_range[1]-D0_range[0]], -elevation[1,-D0_range[1]:D0_range[0]:-1]/50)

elevation_crop=elevation[:,7:953]
side_thickness = elevation_crop[1,-D0_range[1]:D0_range[0]:-1]
top_absorption = top[D1_range[0]:D1_range[1],center]/top_flat[D1_range[0]:D1_range[1],center]

# Fitting
abs_coeffs = px.surface_absorption_calibration(side_thickness, top_absorption) ; 

plt.plot (side_thickness, np.log(top_absorption))
plt.plot (side_thickness[:], side_thickness[:]*abs_coeffs['mu']+abs_coeffs['beta'])


In [ ]:
crop=[40,40+784,210,210+400]
#I0=top_flat[crop[0]:crop[1],crop[2]:crop[3]]
absorption_data=np.log(top[crop[0]:crop[1],crop[2]:crop[3]]/top_flat[crop[0]:crop[1],crop[2]:crop[3]])
surface_profile=px.surface_elevation(absorption_data, abs_coeffs, 6)

sz = surface_profile.shape ; 
relative_surface_profile = surface_profile / np.reshape(np.tile(np.max(surface_profile,axis=1),sz[1]),(sz[1],sz[0])).T

# Matplotlib figure, nicer plotly figure in the next cell. 
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x=np.arange(0,surface_profile.shape[0])*scale_D1
y=np.arange(0,surface_profile.shape[1])*scale_D1
X, Y = np.meshgrid(x, y)
ax.plot_surface(X, Y, surface_profile.T)   

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=[
    go.Surface(x=X, y=Y, z=surface_profile.T, surfacecolor=relative_surface_profile.T)])
fig.update_layout(autosize=False,width=600, height=600, coloraxis=dict(cmax=1, cmin=0.7))